In [ ]:
import tensorflow as tf

print(tf.__version__)

from tensorflow.python.client import device_lib
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0, 1"
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
cpus = tf.config.experimental.list_physical_devices(device_type='CPU')
print(gpus, cpus)
# 设置当前程序的可见设备范围
tf.config.experimental.set_visible_devices(devices=gpus, device_type='GPU')

# 设置仅在需要时申请：
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
    
# 下面的方式是设置Tensorflow固定消耗GPU:0的2GB显存
tf.config.experimental.set_virtual_device_configuration(
    gpus[0],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)]
)

# 获取本地 gpu 资源
# def get_available_gpus():
#   local_device_protos = device_lib.list_local_devices()
#   return [x.name for x in local_device_protos if x.device_type == 'GPU']

# print(get_available_gpus())

In [ ]:
with tf.device('/device:GPU:0'):
    w = tf.constant([[2, -3.4]])
    b = tf.constant([4.2])
    x = tf.random.normal([1000, 2], mean=0, stddev=10)
    e = tf.random.normal([1000, 2], mean=0, stddev=0.1)
    W = tf.Variable(tf.constant([5, 1]))
    B = tf.Variable(tf.constant([1]))
w

In [ ]:
import random
from matplotlib import pyplot as plt
# 线性回归模型， y = 
# 生成数据,生成1000组数据

num_inputs = 2
num_examples = 1000
true_y = tf.matmul(x, tf.transpose(w)) + b
x, true_y
# 读取数据

In [ ]:
def set_figsize(figsize=(3.5, 2.5)):
    plt.rcParams['figure.figsize'] = figsize

set_figsize()
plt.scatter(x[: ,1], true_y, 1)

In [ ]:
class Mymodel():
    def __init__(self):
        self.w = tf.Variable([[2.3, 1.2]])
        self.b = tf.Variable([8.0])
    
    def __call__(self, x):
        self.y = tf.matmul(x , tf.transpose(self.w)) + self.b
        return self.y

In [ ]:
def process(x, y):
    return x, y
    
db = tf.data.Dataset.from_tensor_slices((x, true_y))
db = db.shuffle(100)
db2 = db.map(process)
db_batch = db.batch(32)
# db_iter = next(iter(db_batch))
# print(db_iter[1].shape)

In [ ]:
def loss(predicted_y, desired_y):
    return tf.reduce_mean(tf.square(predicted_y - tf.reshape(desired_y, predicted_y.shape)))


In [ ]:
def train(model, inputs, outputs, learning_rate):
    with tf.GradientTape() as t:
        current_loss = loss(model(inputs), outputs)
        print('loss',current_loss)
    dW, db = t.gradient(current_loss, [model.w, model.b])
    # tf.assign_sub(ref, value, use_locking=None, name=None)，变量 ref 减去 value值，即 ref = ref - value
    model.w.assign_sub(learning_rate * dW)
    model.b.assign_sub(learning_rate * db)


In [ ]:
model = Mymodel()

Ws, bs = [], []
epoches = 20

steps = int(1000/32)
for i in range(epoches):
    for d in iter(db_batch):
#         d = next(iter(db_batch))
        it = d[0]
#         print(it)
        ot = d[1]
#         print(ot)
#         current_loss = loss(model(it), ot)
#         print(current_loss)

        train(model, it, ot, learning_rate=0.001)

    Ws.append(model.w.numpy())
    bs.append(model.b.numpy())
    print('w',(Ws[-1]))
    print('b',(bs[-1]))
#         print('Epoch %2d: W=%1.2f b=%1.2f, loss=%2.5f' %
#               (epoch, Ws[-1], bs[-1], current_loss))

In [ ]:
#     w = tf.constant([[2, -3.4]])
#     b = tf.constant([4.2])
# print(bs)
# plt.plot(epochs, Ws, 'r',
#          epochs, bs, 'b')
# plt.plot([TRUE_W] * len(epochs), 'r--',
#          [TRUE_b] * len(epochs), 'b--')
# plt.legend(['W', 'b', 'true W', 'true_b'])
# plt.show()


plt.figure(figsize=(100, 60))
plt.plot(range(epoches), [i[0] for i in Ws], 'r',
         range(epoches), [i[0] for i in bs], 'b', linewidth=2)
plt.plot([2] * epoches, 'r--',
         [4.2] * epoches, 'b--', linewidth=2)

plt.xlabel('epoches(time)')
plt.ylabel("w/b")
plt.grid(True)
plt.title("Python chart")
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.legend(['W', 'b', 'true W', 'true_b'])
plt.show()